<a href="https://colab.research.google.com/github/Subhamtr01/BBDA/blob/main/Copy_of_4_missing_values_pipelining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
#Last amended: 1st June, 2023
# Ref: https://www.kaggle.com/code/alexisbcook/missing-values
# Data Source: https://www.kaggle.com/competitions/tabular-playground-series-aug-2022/data


This [data represents](https://www.kaggle.com/competitions/tabular-playground-series-aug-2022/data) the results of a large product testing study. For each product_code you are given a number of product attributes (fixed for the code) as well as a number of measurement values for each individual product, representing various lab testing methods. Each product is used in a simulated real-world environment experiment, and and absorbs a certain amount of fluid (loading) to see whether or not it fails.

Your task is to use the data to predict individual product failures of new codes with their individual lab test results.

# Simple Data pipelining

In [114]:
# 1.0

import pandas as pd

# 1.0.1
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

# 1.0.2
from pathlib import Path


In [115]:
# 1.1
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [116]:
# 1.2 Display from a cell outputs of multiple commands:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [117]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [118]:
# 1.2.1 Where is our data:

pathToFolder = "/content/drive/MyDrive/BDDA1"


In [119]:
# 1.2.2 And our file?

path = Path(pathToFolder) / "producttesting.zip"

In [143]:
# 2.1 Load the data
data = pd.read_csv(path)

In [144]:
# 2.2
data.shape  #(26570, 26)
data.head()


(26570, 26)

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [145]:
# 2.2.1 Get target
y = data.pop('failure')


In [146]:
X = data.select_dtypes(exclude = ['object'])
X.pop("id")

0            0
1            1
2            2
3            3
4            4
         ...  
26565    26565
26566    26566
26567    26567
26568    26568
26569    26569
Name: id, Length: 26570, dtype: int64

In [147]:
X = data.select_dtypes(exclude = ['object'])
X

,id,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,0,80.10,9,5,7,8,4,18.040,12.518,15.748,...,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100
1,1,84.89,9,5,14,3,3,18.213,11.540,17.717,...,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057
2,2,82.43,9,5,12,1,5,18.057,11.652,16.738,...,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376
3,3,101.07,9,5,13,2,6,17.295,11.188,18.576,...,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282
4,4,188.06,9,5,9,2,8,19.346,12.950,16.990,...,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,26565,158.95,6,9,6,16,4,16.301,13.259,18.068,...,19.354,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131
26566,26566,146.02,6,9,10,12,8,17.543,NaN,17.984,...,19.563,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924
26567,26567,115.62,6,9,1,10,1,15.670,11.535,16.778,...,19.279,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364
26568,26568,106.38,6,9,2,9,4,18.059,NaN,16.918,...,19.358,11.392,17.064,17.814,14.928,16.273,15.485,13.624,12.865,730.156


In [148]:
si=SimpleImputer(strategy= "median")

In [149]:
column_name =list(X.columns)

In [150]:
si.fit(X)

SimpleImputer(strategy='median')

In [151]:
X=si.transform(X)

In [152]:
import pandas as pd
X = pd.DataFrame(X,columns=column_name)

In [153]:
X.isnull().sum()

id                0
loading           0
attribute_2       0
attribute_3       0
measurement_0     0
measurement_1     0
measurement_2     0
measurement_3     0
measurement_4     0
measurement_5     0
measurement_6     0
measurement_7     0
measurement_8     0
measurement_9     0
measurement_10    0
measurement_11    0
measurement_12    0
measurement_13    0
measurement_14    0
measurement_15    0
measurement_16    0
measurement_17    0
dtype: int64

In [154]:
 rf = RandomForestClassifier ()

In [155]:
rf = rf.fit(X,y)

In [156]:
rf.predict(X)

array([0, 0, 0, ..., 0, 0, 0])

In [157]:
s=rf.predict(X)

In [162]:
(s==y).sum()/X.shape

array([1.00000000e+00, 1.20772727e+03])

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.25)

In [165]:
 rf = RandomForestClassifier ()

In [166]:
rf = rf.fit(X,y)

In [167]:
rf.predict(X)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
pipe = make_pipeline(SimpleImputer(strategy = 'mean'), RandomForestClassifier())

In [ ]:
# 3.0 Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
pred = pipe.predict(X_valid)

In [ ]:
accu = accuracy_score(y_valid,pred)
accu   # 79%

# Your Turn

a. Add Standard Scaler and PCA to it  
b. Add cross validation to it

In [ ]:
### ----------------------- Done -----------------